In [1]:
import torch
import torchaudio

################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################

/root/anaconda3/lib/python3.7/site-packages/torchaudio/backend/utils.py:54: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  '"sox" backend is being deprecated. '


In [2]:
! nvidia-smi

Wed Jun 30 15:00:08 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.116.00   Driver Version: 418.116.00   CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM3...  On   | 00000000:59:00.0 Off |                    0 |
| N/A   40C    P0    79W / 350W |   4350MiB / 32480MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [3]:
import numpy as np
np_rng = np.random.default_rng(1)
import pandas as pd


import urllib.parse
from IPython.display import display, Markdown

import os

from lidbox.meta import (
    common_voice,
    generate_label2target,
    verify_integrity,
    read_audio_durations,
    random_oversampling_on_split
)


train = pd.read_csv("/tf/datasets/train.tsv", sep="\t")
test = pd.read_csv("/tf/datasets/test.tsv", sep="\t")
dev = pd.read_csv("/tf/datasets/dev.tsv", sep="\t")

train["path"] = train["path"].apply(lambda x: x[:-3] + "mp3")
test["path"] = test["path"].apply(lambda x: x[:-3] + "mp3")
dev["path"] = dev["path"].apply(lambda x: x[:-3] + "mp3")

train["split"] = "train"
test["split"] = "test"
dev["split"] = "dev"
#test = test.sample(30000, replace=False)
meta = pd.concat([train, test, dev])


In [4]:
meta.loc[meta["locale"] != "kz", "path"] = "/tf/datasets/data_untar/cv-corpus-6.1-2020-12-11/" +  meta.loc[meta["locale"] != "kz"]["locale"] + "/clips/" + meta.loc[meta["locale"] != "kz"]["path"]
targets = {"kz": 0, "ru": 1, "en":2, "other":3}
meta["target"] = meta["locale"]
meta.loc[(meta["locale"] != "kz") & (meta["locale"] != "ru") & (meta["locale"]!="en"), "target"] = "other"
meta = meta.loc[meta["path"] != "/tf/datasets/data_untar/cv-corpus-6.1-2020-12-11/kz/clips/5f590a130a73c.mp3"]
meta = meta.loc[meta["path"] != "/tf/datasets/data_untar/cv-corpus-6.1-2020-12-11/kz/clips/5ef9bd9ba7029.mp3"]

meta["id"] = meta["Unnamed: 0"].apply(str)
meta["target"] = meta["target"].map(targets)

meta

2021-06-30 15:00:09.749 I numexpr.utils: Note: detected 96 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
2021-06-30 15:00:09.750 I numexpr.utils: Note: NumExpr detected 96 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2021-06-30 15:00:09.750 I numexpr.utils: NumExpr defaulting to 8 threads.


,Unnamed: 0,path,locale,split,target,id
0,1486,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...,ru,train,1,1486
1,56701,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...,kz,train,0,56701
2,3364,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...,ru,train,1,3364
3,110475,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...,rw,train,3,110475
4,45384,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...,en,train,2,45384
...,...,...,...,...,...,...
30751,2677,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...,ru,dev,1,2677
30752,881,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...,ru,dev,1,881
30753,68709,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...,kz,dev,0,68709
30754,249025,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...,en,dev,2,249025


In [5]:
from torch.utils.data import Dataset
import random
import math
DEVICE = 'cuda'
def _get_sample(path, resample=None):
  effects = [
    ["remix", "1"]
  ]
  if resample:
    effects.append(["rate", f'{resample}'])
  return torchaudio.sox_effects.apply_effects_file(path, effects=effects)

SAMPLE_RIR_PATH = os.path.join(os.getcwd(), "rir.wav")

def get_rir_sample(*, resample=None, processed=False):
    rir_raw, sample_rate = _get_sample(SAMPLE_RIR_PATH, resample=resample)
    if not processed:
        return rir_raw, sample_rate
    rir = rir_raw[:, int(sample_rate*1.01):int(sample_rate*1.3)]
    rir = rir / torch.norm(rir, p=2)
    rir = torch.flip(rir, [1])
    return rir, sample_rate

class AudiosDataset(Dataset):
    def __init__(self, paths=None, targets=None, augment=False) -> None:
        self.paths = paths
        self.targets = targets
        self.augment = augment
        self.extractor = torch.hub.load('s3prl/s3prl', 'audio_albert').to(DEVICE)
        self.rir = get_rir_sample()[0]
        
    
    def __len__(self) -> int:
        return len(self.paths)

    def __getitem__(self, idx: int) -> dict:
        y, sr = torchaudio.load(self.paths.iloc[idx], normalization=True)
        """
        if self.augment:
            effects = [
                    ["lowpass", "-1", "300"], 
                    ["speed", f"{random.uniform(0.7, 1.3)}"],  # change speed
                  ]
            y, sr = torchaudio.sox_effects.apply_effects_tensor(
                y, sr, effects)
        #
            
        if self.augment:
            # augment sound in order to imitate the room change
            rir = self.rir[:, int(16000*1.1):int(16000*1.3)]
            rir = rir / torch.norm(rir, p=2)
            rir = torch.flip(rir, [1])
            y = torch.nn.functional.conv1d(y[None, ...], rir[None, ...])[0]
        
        """
        y = torchaudio.transforms.Resample(orig_freq=sr, new_freq=16000)(y)
        y = torchaudio.transforms.Vad(sample_rate = 16000)(y)

        """
        if self.augment:
            y = (0.5)*torch.randn(y.shape)
        """
        fixed_length = 1024
        with torch.no_grad():
            representation = self.extractor(y.to(DEVICE))[0]
        # returning result
        representation = representation.permute(1, 0)
        if representation.shape[1] < fixed_length:
            representation = torch.nn.functional.pad(
              representation, (0, fixed_length - representation.shape[1]))
        else:
            representation = representation[:, :fixed_length]
        representation = representation.permute(1, 0)
        result = {"target":self.targets.iloc[idx], "representation":representation}

        return result

In [6]:
ds = AudiosDataset(meta["path"], meta["target"])

Using cache found in /root/.cache/torch/hub/s3prl_s3prl_master


[hubconf] can not import upstream.byol_a.hubconf: No module named 'easydict'... Pass.
[hubconf] can not import upstream.decoar.hubconf: No module named 'mxnet'... Pass.
[hubconf] can not import upstream.wav2vec2_hug.hubconf: No module named 'transformers'... Pass.
[hubconf] can not import upstream.pase.hubconf: No module named 'pase'... Pass.
[hubconf] can not import upstream.byol_a.hubconf: No module named 'easydict'... Pass.
[hubconf] can not import upstream.decoar.hubconf: No module named 'mxnet'... Pass.
[hubconf] can not import upstream.wav2vec2_hug.hubconf: No module named 'transformers'... Pass.
[hubconf] can not import upstream.pase.hubconf: No module named 'pase'... Pass.
2021-06-30 15:00:12.861 I filelock: Lock 140125655932240 acquired on /root/.cache/torch/hub/s3prl_cache/55805839eb58b56972be4a9994ca99c862d2572cfd74ae26c916e8ced5d45a7a.lock
Using cache found in /root/.cache/torch/hub/s3prl_cache/55805839eb58b56972be4a9994ca99c862d2572cfd74ae26c916e8ced5d45a7a
for https://www

/root/anaconda3/lib/python3.7/site-packages/torchaudio/functional.py:318: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  "At least one mel filterbank has all zero values. "
/root/anaconda3/lib/python3.7/site-packages/torch/functional.py:516: UserWarning: stft will require the return_complex parameter be explicitly  specified in a future PyTorch release. Use return_complex=False  to preserve the current behavior or return_complex=True to return  a complex output. (Triggered internally at  /pytorch/aten/src/ATen/native/SpectralOps.cpp:653.)
  normalized, onesided, return_complex)
/root/anaconda3/lib/python3.7/site-packages/torch/functional.py:516: UserWarning: The function torch.rfft is deprecated and will be removed in a future PyTorch release. Use the new torch.fft module functions, instead, by importing torch.fft and calling torch.fft.fft or torch.fft.rfft. (Trigger

In [7]:
from torch.utils.data import TensorDataset, DataLoader
train_ds = AudiosDataset(meta.loc[meta["split"]=="train"]["path"], meta.loc[meta["split"]=="train"]["target"], augment=True)
val_ds = AudiosDataset(meta.loc[meta["split"]=="dev"]["path"], meta.loc[meta["split"]=="dev"]["target"])
test_ds = AudiosDataset(meta.loc[meta["split"]=="test"]["path"], meta.loc[meta["split"]=="test"]["target"])

[hubconf] can not import upstream.byol_a.hubconf: No module named 'easydict'... Pass.
[hubconf] can not import upstream.decoar.hubconf: No module named 'mxnet'... Pass.
[hubconf] can not import upstream.wav2vec2_hug.hubconf: No module named 'transformers'... Pass.
[hubconf] can not import upstream.pase.hubconf: No module named 'pase'... Pass.
2021-06-30 15:00:17.824 I filelock: Lock 140125636039568 acquired on /root/.cache/torch/hub/s3prl_cache/55805839eb58b56972be4a9994ca99c862d2572cfd74ae26c916e8ced5d45a7a.lock
Using cache found in /root/.cache/torch/hub/s3prl_cache/55805839eb58b56972be4a9994ca99c862d2572cfd74ae26c916e8ced5d45a7a
for https://www.dropbox.com/s/3wgynxmod77ha1z/states-1000000.ckpt?dl=0
2021-06-30 15:00:17.825 I filelock: Lock 140125636039568 released on /root/.cache/torch/hub/s3prl_cache/55805839eb58b56972be4a9994ca99c862d2572cfd74ae26c916e8ced5d45a7a.lock


Using cache found in /root/.cache/torch/hub/s3prl_s3prl_master


[UpstreamExpert] - Using the default upstream expert config
[hubconf] can not import upstream.byol_a.hubconf: No module named 'easydict'... Pass.
[hubconf] can not import upstream.decoar.hubconf: No module named 'mxnet'... Pass.
[hubconf] can not import upstream.wav2vec2_hug.hubconf: No module named 'transformers'... Pass.
[hubconf] can not import upstream.pase.hubconf: No module named 'pase'... Pass.
2021-06-30 15:00:18.082 I filelock: Lock 140125636040144 acquired on /root/.cache/torch/hub/s3prl_cache/55805839eb58b56972be4a9994ca99c862d2572cfd74ae26c916e8ced5d45a7a.lock
Using cache found in /root/.cache/torch/hub/s3prl_cache/55805839eb58b56972be4a9994ca99c862d2572cfd74ae26c916e8ced5d45a7a
for https://www.dropbox.com/s/3wgynxmod77ha1z/states-1000000.ckpt?dl=0
2021-06-30 15:00:18.083 I filelock: Lock 140125636040144 released on /root/.cache/torch/hub/s3prl_cache/55805839eb58b56972be4a9994ca99c862d2572cfd74ae26c916e8ced5d45a7a.lock


Using cache found in /root/.cache/torch/hub/s3prl_s3prl_master


[UpstreamExpert] - Using the default upstream expert config
[hubconf] can not import upstream.byol_a.hubconf: No module named 'easydict'... Pass.
[hubconf] can not import upstream.decoar.hubconf: No module named 'mxnet'... Pass.
[hubconf] can not import upstream.wav2vec2_hug.hubconf: No module named 'transformers'... Pass.
[hubconf] can not import upstream.pase.hubconf: No module named 'pase'... Pass.
2021-06-30 15:00:18.336 I filelock: Lock 140125635737616 acquired on /root/.cache/torch/hub/s3prl_cache/55805839eb58b56972be4a9994ca99c862d2572cfd74ae26c916e8ced5d45a7a.lock
Using cache found in /root/.cache/torch/hub/s3prl_cache/55805839eb58b56972be4a9994ca99c862d2572cfd74ae26c916e8ced5d45a7a
for https://www.dropbox.com/s/3wgynxmod77ha1z/states-1000000.ckpt?dl=0
2021-06-30 15:00:18.336 I filelock: Lock 140125635737616 released on /root/.cache/torch/hub/s3prl_cache/55805839eb58b56972be4a9994ca99c862d2572cfd74ae26c916e8ced5d45a7a.lock


Using cache found in /root/.cache/torch/hub/s3prl_s3prl_master


[UpstreamExpert] - Using the default upstream expert config


In [8]:
from torch.utils.data import Dataset
import random
import math
import torch
DEVICE = 'cuda'
def _get_sample(path, resample=None):
  effects = [
    ["remix", "1"]
  ]
  if resample:
    effects.append(["rate", f'{resample}'])
  return torchaudio.sox_effects.apply_effects_file(path, effects=effects)

SAMPLE_RIR_PATH = os.path.join(os.getcwd(), "rir.wav")

def get_rir_sample(*, resample=None, processed=False):
    rir_raw, sample_rate = _get_sample(SAMPLE_RIR_PATH, resample=resample)
    if not processed:
        return rir_raw, sample_rate
    rir = rir_raw[:, int(sample_rate*1.01):int(sample_rate*1.3)]
    rir = rir / torch.norm(rir, p=2)
    rir = torch.flip(rir, [1])
    return rir, sample_rate

class AudiosDataset(Dataset):
    def __init__(self, paths=None, targets=None, augment=False) -> None:
        self.paths = paths
        self.targets = targets
        self.augment = augment
        self.extractor = torch.hub.load('s3prl/s3prl', 'audio_albert').to(DEVICE)
        self.rir = get_rir_sample()[0]
        
    
    def __len__(self) -> int:
        return len(self.paths)

    def __getitem__(self, idx: int) -> dict:
        y, sr = torchaudio.load(self.paths.iloc[idx], normalization=True)
        """
        if self.augment:
            effects = [
                    ["lowpass", "-1", "300"], 
                    ["speed", f"{random.uniform(0.7, 1.3)}"],  # change speed
                  ]
            y, sr = torchaudio.sox_effects.apply_effects_tensor(
                y, sr, effects)
        #
            
        if self.augment:
            # augment sound in order to imitate the room change
            rir = self.rir[:, int(16000*1.1):int(16000*1.3)]
            rir = rir / torch.norm(rir, p=2)
            rir = torch.flip(rir, [1])
            y = torch.nn.functional.conv1d(y[None, ...], rir[None, ...])[0]
        
        """
        y = torchaudio.transforms.Resample(orig_freq=sr, new_freq=16000)(y)
        y = torchaudio.transforms.Vad(sample_rate = 16000)(y)

        """
        if self.augment:
            y = (0.5)*torch.randn(y.shape)
        """
        fixed_length = 16000 * 12
        
        # returning result
        if y.shape[1] < fixed_length:
            y = torch.nn.functional.pad(
              y, (0, fixed_length - y.shape[1]))
        else:
            y = y[:, :fixed_length]
        result = {"target":self.targets.iloc[idx], "representation":y}

        return result

In [9]:
ds = AudiosDataset(meta["path"], meta["target"])
for i in ds:
    print(i['representation'].shape)
    break

[hubconf] can not import upstream.byol_a.hubconf: No module named 'easydict'... Pass.
[hubconf] can not import upstream.decoar.hubconf: No module named 'mxnet'... Pass.
[hubconf] can not import upstream.wav2vec2_hug.hubconf: No module named 'transformers'... Pass.
[hubconf] can not import upstream.pase.hubconf: No module named 'pase'... Pass.
2021-06-30 15:00:18.562 I filelock: Lock 140125635891472 acquired on /root/.cache/torch/hub/s3prl_cache/55805839eb58b56972be4a9994ca99c862d2572cfd74ae26c916e8ced5d45a7a.lock
Using cache found in /root/.cache/torch/hub/s3prl_cache/55805839eb58b56972be4a9994ca99c862d2572cfd74ae26c916e8ced5d45a7a
for https://www.dropbox.com/s/3wgynxmod77ha1z/states-1000000.ckpt?dl=0
2021-06-30 15:00:18.563 I filelock: Lock 140125635891472 released on /root/.cache/torch/hub/s3prl_cache/55805839eb58b56972be4a9994ca99c862d2572cfd74ae26c916e8ced5d45a7a.lock


Using cache found in /root/.cache/torch/hub/s3prl_s3prl_master


[UpstreamExpert] - Using the default upstream expert config
torch.Size([1, 192000])


In [10]:
from torch.utils.data import TensorDataset, DataLoader
train_ds = AudiosDataset(meta.loc[meta["split"]=="train"]["path"], meta.loc[meta["split"]=="train"]["target"], augment=True)
val_ds = AudiosDataset(meta.loc[meta["split"]=="dev"]["path"], meta.loc[meta["split"]=="dev"]["target"])
test_ds = AudiosDataset(meta.loc[meta["split"]=="test"]["path"], meta.loc[meta["split"]=="test"]["target"])

[hubconf] can not import upstream.byol_a.hubconf: No module named 'easydict'... Pass.
[hubconf] can not import upstream.decoar.hubconf: No module named 'mxnet'... Pass.
[hubconf] can not import upstream.wav2vec2_hug.hubconf: No module named 'transformers'... Pass.
[hubconf] can not import upstream.pase.hubconf: No module named 'pase'... Pass.
2021-06-30 15:00:19.085 I filelock: Lock 140125635957072 acquired on /root/.cache/torch/hub/s3prl_cache/55805839eb58b56972be4a9994ca99c862d2572cfd74ae26c916e8ced5d45a7a.lock
Using cache found in /root/.cache/torch/hub/s3prl_cache/55805839eb58b56972be4a9994ca99c862d2572cfd74ae26c916e8ced5d45a7a
for https://www.dropbox.com/s/3wgynxmod77ha1z/states-1000000.ckpt?dl=0
2021-06-30 15:00:19.086 I filelock: Lock 140125635957072 released on /root/.cache/torch/hub/s3prl_cache/55805839eb58b56972be4a9994ca99c862d2572cfd74ae26c916e8ced5d45a7a.lock
[UpstreamExpert] - Using the default upstream expert config


Using cache found in /root/.cache/torch/hub/s3prl_s3prl_master


[hubconf] can not import upstream.byol_a.hubconf: No module named 'easydict'... Pass.
[hubconf] can not import upstream.decoar.hubconf: No module named 'mxnet'... Pass.
[hubconf] can not import upstream.wav2vec2_hug.hubconf: No module named 'transformers'... Pass.
[hubconf] can not import upstream.pase.hubconf: No module named 'pase'... Pass.
2021-06-30 15:00:19.352 I filelock: Lock 140125656593744 acquired on /root/.cache/torch/hub/s3prl_cache/55805839eb58b56972be4a9994ca99c862d2572cfd74ae26c916e8ced5d45a7a.lock
Using cache found in /root/.cache/torch/hub/s3prl_cache/55805839eb58b56972be4a9994ca99c862d2572cfd74ae26c916e8ced5d45a7a
for https://www.dropbox.com/s/3wgynxmod77ha1z/states-1000000.ckpt?dl=0
2021-06-30 15:00:19.353 I filelock: Lock 140125656593744 released on /root/.cache/torch/hub/s3prl_cache/55805839eb58b56972be4a9994ca99c862d2572cfd74ae26c916e8ced5d45a7a.lock


Using cache found in /root/.cache/torch/hub/s3prl_s3prl_master


[UpstreamExpert] - Using the default upstream expert config
[hubconf] can not import upstream.byol_a.hubconf: No module named 'easydict'... Pass.
[hubconf] can not import upstream.decoar.hubconf: No module named 'mxnet'... Pass.
[hubconf] can not import upstream.wav2vec2_hug.hubconf: No module named 'transformers'... Pass.
[hubconf] can not import upstream.pase.hubconf: No module named 'pase'... Pass.
2021-06-30 15:00:19.597 I filelock: Lock 140125635959120 acquired on /root/.cache/torch/hub/s3prl_cache/55805839eb58b56972be4a9994ca99c862d2572cfd74ae26c916e8ced5d45a7a.lock
Using cache found in /root/.cache/torch/hub/s3prl_cache/55805839eb58b56972be4a9994ca99c862d2572cfd74ae26c916e8ced5d45a7a
for https://www.dropbox.com/s/3wgynxmod77ha1z/states-1000000.ckpt?dl=0
2021-06-30 15:00:19.598 I filelock: Lock 140125635959120 released on /root/.cache/torch/hub/s3prl_cache/55805839eb58b56972be4a9994ca99c862d2572cfd74ae26c916e8ced5d45a7a.lock


Using cache found in /root/.cache/torch/hub/s3prl_s3prl_master


[UpstreamExpert] - Using the default upstream expert config


In [11]:
batch_size = 8
num_workers = 10
loaders = {
    "train": DataLoader(
        train_ds,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        drop_last=True,
    ),
    "valid": DataLoader(
        val_ds,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        drop_last=True,
    ),
    "test":DataLoader(
        test_ds,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        drop_last=True,
    ),
}

In [12]:
m = torch.hub.load('s3prl/s3prl', 'audio_albert').to(DEVICE)

[hubconf] can not import upstream.byol_a.hubconf: No module named 'easydict'... Pass.
[hubconf] can not import upstream.decoar.hubconf: No module named 'mxnet'... Pass.
[hubconf] can not import upstream.wav2vec2_hug.hubconf: No module named 'transformers'... Pass.
[hubconf] can not import upstream.pase.hubconf: No module named 'pase'... Pass.
2021-06-30 15:00:19.811 I filelock: Lock 140129131183568 acquired on /root/.cache/torch/hub/s3prl_cache/55805839eb58b56972be4a9994ca99c862d2572cfd74ae26c916e8ced5d45a7a.lock
Using cache found in /root/.cache/torch/hub/s3prl_cache/55805839eb58b56972be4a9994ca99c862d2572cfd74ae26c916e8ced5d45a7a
for https://www.dropbox.com/s/3wgynxmod77ha1z/states-1000000.ckpt?dl=0
2021-06-30 15:00:19.812 I filelock: Lock 140129131183568 released on /root/.cache/torch/hub/s3prl_cache/55805839eb58b56972be4a9994ca99c862d2572cfd74ae26c916e8ced5d45a7a.lock
[UpstreamExpert] - Using the default upstream expert config


Using cache found in /root/.cache/torch/hub/s3prl_s3prl_master


In [13]:
from torch import nn
import torch.nn.functional as F
class Classifier(nn.Module):
    def __init__(self, extractor):
        super().__init__()
        self.extractor = extractor
        self.rnn = nn.LSTM(input_size=768, hidden_size=512, num_layers=5, bidirectional=True, batch_first=True, dropout=0.7)
        self.classifier = nn.Sequential(
                    nn.Linear(1024, 512),
                    nn.ReLU(),
                    nn.BatchNorm1d(512),
                    nn.Dropout(0.5),
                    nn.Linear(512, 4),
                    )
    def forward(self, x):
        features = self.extractor(torch.squeeze(x))
        features = torch.stack(features)
        res, _ = self.rnn(features)
        res = self.classifier(res[:, -1, :])
        return F.log_softmax(res, dim=1)

In [14]:
model = Classifier(m)

In [16]:
from tqdm import tqdm

def train(model, opt, scheduler, loss_fn, epochs, data_tr, data_val, max_stable=5):
    best_val_loss = 1e9
    counter = 0
    for epoch in range(epochs):
        #tic = time()
        print('* Epoch %d/%d' % (epoch+1, epochs))

        avg_loss = 0
        model.train()  # train mode
        for batch in tqdm(data_tr):
            loss = 0
            # data to device
            X_batch, Y_batch = batch["representation"], batch["target"]
            #print(X_batch.shape)
            X_batch = X_batch.to(DEVICE)
            Y_batch = Y_batch.to(DEVICE)
            # set parameter gradients to zero
            opt.zero_grad()
            # forward
            Y_pred = model(X_batch)
            #print(Y_pred)
            loss = loss_fn(Y_pred, Y_batch)# forward-pass
            loss.backward()  # backward-pass
            opt.step()  # update weights
            if not scheduler is None:
                scheduler.step()
            # calculate loss to show the user
            avg_loss += loss / len(data_tr)
      #  toc = time()
        print('loss: %f' % avg_loss)
        # show intermediate results
        model.eval()  # testing mode
        val_loss = 0
        print("start validation")
        for v_b in tqdm(data_val):
            X_val, Y_val = v_b["representation"], v_b["target"]
            Y_hat = model(X_val.to(DEVICE)).detach().cpu()# detach and put into cpu
            val_loss += loss_fn(Y_hat, Y_val)
        val_loss /= len(data_val)
        print( f"validation loss: {val_loss}")
        if val_loss <= best_val_loss and val_loss > 0:
            counter = 0
            print("Save new model!")
            best_val_loss = val_loss
            torch.save(model.state_dict(), f'best_model.h5')
        else:
            counter += 1
        if counter == max_stable:
            break

In [18]:
model.load_state_dict(torch.load('best_model.h5'))

<All keys matched successfully>

In [17]:
DEVICE = 'cuda'
max_epochs = 100
model = model.to(DEVICE)
#torch.cuda.empty_cache()
loss_fn =  nn.CrossEntropyLoss()
optimaizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimaizer, T_0=5, T_mult=1, eta_min=1e-8, last_epoch=-1)
train(model, optimaizer,scheduler, loss_fn, max_epochs, loaders["train"], loaders["valid"])

  0%|          | 0/10306 [00:00<?, ?it/s]

* Epoch 1/100


  0%|          | 0/10306 [00:52<?, ?it/s]


KeyboardInterrupt: 

In [20]:
true_labels = []
predicted_labels = []
for batch in tqdm(loaders["test"]):
    model.eval()
    prediction = model(batch["representation"].to(DEVICE)).detach().cpu()
    predicted_labels.extend(torch.argmax(prediction, dim=1).tolist())
    true_labels.extend(batch["target"].tolist())


100%|██████████| 6971/6971 [54:59<00:00,  2.11it/s]  


In [21]:
from sklearn.metrics import classification_report

report = classification_report(true_labels, predicted_labels, target_names=list(targets.keys()), labels=range(4))
print(report)



              precision    recall  f1-score   support

          kz       1.00      1.00      1.00     17341
          ru       0.95      0.84      0.89     10379
          en       0.93      0.93      0.93     12964
       other       0.85      0.92      0.88     15084

    accuracy                           0.93     55768
   macro avg       0.93      0.92      0.93     55768
weighted avg       0.93      0.93      0.93     55768



In [23]:
from sklearn.metrics import classification_report

report = classification_report(true_labels, predicted_labels, target_names=list(targets.keys()), labels=range(4))
print(report)



              precision    recall  f1-score   support

          kz       1.00      1.00      1.00     17337
          ru       0.67      0.33      0.44     10374
          en       0.65      0.18      0.29     12956
       other       0.45      0.89      0.60     15077

    accuracy                           0.66     55744
   macro avg       0.69      0.60      0.58     55744
weighted avg       0.71      0.66      0.62     55744



##  Vox test

In [22]:
import numpy as np
np_rng = np.random.default_rng(1)


import urllib.parse
from IPython.display import display, Markdown

import os

from lidbox.meta import (
    common_voice,
    generate_label2target,
    verify_integrity,
    read_audio_durations,
    random_oversampling_on_split
)


test = pd.read_csv("/tf/datasets/new_test.tsv", sep="\t")

test["path"] = test["path"].apply(lambda x: x[:-3] + "mp3")

test["split"] = "test"
meta = pd.concat([test])


In [23]:
meta.loc[((meta["locale"] != "kz") & ~(((meta["split"] == "dev") | (meta["split"] == "test")) & ((meta["locale"] == "ru") | (meta["locale"] == "kz") | (meta["locale"] == "en")))), "path"] = "/tf/datasets/data_untar/cv-corpus-6.1-2020-12-11/" + meta.loc[((meta["locale"] != "kz") & ~(((meta["split"] == "dev") | (meta["split"] == "test")) & ((meta["locale"] == "ru") | (meta["locale"] == "kz") | (meta["locale"] == "en"))))]["locale"]  + "/clips/" + meta.loc[((meta["locale"] != "kz") & ~(((meta["split"] == "dev") | (meta["split"] == "test")) & ((meta["locale"] == "ru") | (meta["locale"] == "kz") | (meta["locale"] == "en"))))]["path"]
targets = {"kz": 0, "ru": 1, "en":2, "other":3}
meta["target"] = meta["locale"]
meta.loc[(meta["locale"] != "kz") & (meta["locale"] != "ru") & (meta["locale"]!="en"), "target"] = "other"
meta = meta.loc[meta["path"] != "/tf/datasets/data_untar/cv-corpus-6.1-2020-12-11/kz/clips/5f590a130a73c.mp3"]
meta = meta.loc[meta["path"] != "/tf/datasets/data_untar/cv-corpus-6.1-2020-12-11/kz/clips/5ef9bd9ba7029.mp3"]

meta["id"] = meta["Unnamed: 0"].apply(str)
meta["target"] = meta["target"].map(targets)


workdir = "/tf/datasets/transformer"


In [24]:
meta.loc[(meta["split"] == "test") & (meta["locale"] == "ru"), "path"] = meta.loc[(meta["split"] == "test") & (meta["locale"] == "ru")]["path"].apply(lambda x: f"/tf/datasets/vox/ru_test/{x}")
meta.loc[(meta["split"] == "test") & (meta["locale"] == "ru"), "path"]
meta.loc[(meta["split"] == "test") & (meta["locale"] == "kz"), "path"] = meta.loc[(meta["split"] == "test") & (meta["locale"] == "kz")]["path"].apply(lambda x: f"/tf/datasets/vox/kz_test/{x}")
meta.loc[(meta["split"] == "test") & (meta["locale"] == "kz"), "path"] 
meta.loc[(meta["split"] == "test") & (meta["locale"] == "en"), "path"] = meta.loc[(meta["split"] == "test") & (meta["locale"] == "en")]["path"].apply(lambda x: f"/tf/datasets/vox/en_test/{x}")
meta.loc[(meta["split"] == "test") & (meta["locale"] == "en"), "path"] 

0       /tf/datasets/vox/en_test/shrDRhToGpY__U__S133-...
1       /tf/datasets/vox/en_test/mzfg0RGJnV8__U__S123-...
2       /tf/datasets/vox/en_test/-_PPCH3y0eE__U__S1---...
3       /tf/datasets/vox/en_test/DQMxvGYyu6Q__U__S0---...
4       /tf/datasets/vox/en_test/x4lfSc7PrB0__U__S0---...
                              ...                        
9995    /tf/datasets/vox/en_test/KLiy94kfZI4__U__S133-...
9996    /tf/datasets/vox/en_test/YTlliEr5LOA__U__S113-...
9997    /tf/datasets/vox/en_test/bSs0gNq6Kkc__U__S0---...
9998    /tf/datasets/vox/en_test/Da7c-BY6MDA__U__S2---...
9999    /tf/datasets/vox/en_test/VWvPndMo1F8__U__S24--...
Name: path, Length: 10000, dtype: object

In [25]:
meta.loc[meta["split"]=="test", "Unnamed: 0"] = meta.loc[meta["split"]=="test"]["path"]

In [26]:
meta["id"] = meta["Unnamed: 0"].apply(str)

In [27]:
meta.loc[meta["split"] == "test", "id"] = meta.loc[meta["split"] == "test"]["path"]

In [28]:
meta = meta.set_index("Unnamed: 0")
meta.loc[meta["split"]=="test"]

,path,locale,split,target,id
Unnamed: 0,,,,,
/tf/datasets/vox/en_test/shrDRhToGpY__U__S133---0944.430-0958.260.mp3,/tf/datasets/vox/en_test/shrDRhToGpY__U__S133-...,en,test,2,/tf/datasets/vox/en_test/shrDRhToGpY__U__S133-...
/tf/datasets/vox/en_test/mzfg0RGJnV8__U__S123---0427.020-0444.670.mp3,/tf/datasets/vox/en_test/mzfg0RGJnV8__U__S123-...,en,test,2,/tf/datasets/vox/en_test/mzfg0RGJnV8__U__S123-...
/tf/datasets/vox/en_test/-_PPCH3y0eE__U__S1---0398.760-0403.940.mp3,/tf/datasets/vox/en_test/-_PPCH3y0eE__U__S1---...,en,test,2,/tf/datasets/vox/en_test/-_PPCH3y0eE__U__S1---...
/tf/datasets/vox/en_test/DQMxvGYyu6Q__U__S0---1473.480-1485.720.mp3,/tf/datasets/vox/en_test/DQMxvGYyu6Q__U__S0---...,en,test,2,/tf/datasets/vox/en_test/DQMxvGYyu6Q__U__S0---...
/tf/datasets/vox/en_test/x4lfSc7PrB0__U__S0---0125.230-0140.900.mp3,/tf/datasets/vox/en_test/x4lfSc7PrB0__U__S0---...,en,test,2,/tf/datasets/vox/en_test/x4lfSc7PrB0__U__S0---...
...,...,...,...,...,...
/tf/datasets/data_untar/cv-corpus-6.1-2020-12-11/it/clips/common_voice_it_20015623.mp3,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...,it,test,3,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...
/tf/datasets/data_untar/cv-corpus-6.1-2020-12-11/uk/clips/common_voice_uk_23554602.mp3,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...,uk,test,3,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...
/tf/datasets/data_untar/cv-corpus-6.1-2020-12-11/tr/clips/common_voice_tr_20416266.mp3,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...,tr,test,3,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...


In [29]:
meta.loc[meta["split"] == "test"] = meta.loc[(meta["split"] == "test") & (meta["target"] != 3)] 

In [30]:
meta.loc[meta["split"]=="test", "id"] = meta.loc[meta["split"]=="test"]["path"]
meta.loc[meta["split"]=="test"] 

,path,locale,split,target,id
Unnamed: 0,,,,,
/tf/datasets/vox/en_test/shrDRhToGpY__U__S133---0944.430-0958.260.mp3,/tf/datasets/vox/en_test/shrDRhToGpY__U__S133-...,en,test,2.0,/tf/datasets/vox/en_test/shrDRhToGpY__U__S133-...
/tf/datasets/vox/en_test/mzfg0RGJnV8__U__S123---0427.020-0444.670.mp3,/tf/datasets/vox/en_test/mzfg0RGJnV8__U__S123-...,en,test,2.0,/tf/datasets/vox/en_test/mzfg0RGJnV8__U__S123-...
/tf/datasets/vox/en_test/-_PPCH3y0eE__U__S1---0398.760-0403.940.mp3,/tf/datasets/vox/en_test/-_PPCH3y0eE__U__S1---...,en,test,2.0,/tf/datasets/vox/en_test/-_PPCH3y0eE__U__S1---...
/tf/datasets/vox/en_test/DQMxvGYyu6Q__U__S0---1473.480-1485.720.mp3,/tf/datasets/vox/en_test/DQMxvGYyu6Q__U__S0---...,en,test,2.0,/tf/datasets/vox/en_test/DQMxvGYyu6Q__U__S0---...
/tf/datasets/vox/en_test/x4lfSc7PrB0__U__S0---0125.230-0140.900.mp3,/tf/datasets/vox/en_test/x4lfSc7PrB0__U__S0---...,en,test,2.0,/tf/datasets/vox/en_test/x4lfSc7PrB0__U__S0---...
...,...,...,...,...,...
/tf/datasets/vox/kz_test/rCpb0p_lyxI__U__S25---0107.830-0127.780.mp3,/tf/datasets/vox/kz_test/rCpb0p_lyxI__U__S25--...,kz,test,0.0,/tf/datasets/vox/kz_test/rCpb0p_lyxI__U__S25--...
/tf/datasets/vox/kz_test/BkLVX9wf2YI__U__S26---0236.830-0241.550.mp3,/tf/datasets/vox/kz_test/BkLVX9wf2YI__U__S26--...,kz,test,0.0,/tf/datasets/vox/kz_test/BkLVX9wf2YI__U__S26--...
/tf/datasets/vox/kz_test/RqdH-JD8TpM__U__S78---0466.720-0470.860.mp3,/tf/datasets/vox/kz_test/RqdH-JD8TpM__U__S78--...,kz,test,0.0,/tf/datasets/vox/kz_test/RqdH-JD8TpM__U__S78--...


In [31]:
newtest_ds = AudiosDataset(meta.loc[meta["split"]=="test"]["path"], meta.loc[meta["split"]=="test"]["target"])

[hubconf] can not import upstream.byol_a.hubconf: No module named 'easydict'... Pass.
[hubconf] can not import upstream.decoar.hubconf: No module named 'mxnet'... Pass.
[hubconf] can not import upstream.wav2vec2_hug.hubconf: No module named 'transformers'... Pass.
[hubconf] can not import upstream.pase.hubconf: No module named 'pase'... Pass.
2021-06-30 16:01:49.417 I filelock: Lock 140119416137232 acquired on /root/.cache/torch/hub/s3prl_cache/55805839eb58b56972be4a9994ca99c862d2572cfd74ae26c916e8ced5d45a7a.lock
Using cache found in /root/.cache/torch/hub/s3prl_cache/55805839eb58b56972be4a9994ca99c862d2572cfd74ae26c916e8ced5d45a7a
for https://www.dropbox.com/s/3wgynxmod77ha1z/states-1000000.ckpt?dl=0
2021-06-30 16:01:49.418 I filelock: Lock 140119416137232 released on /root/.cache/torch/hub/s3prl_cache/55805839eb58b56972be4a9994ca99c862d2572cfd74ae26c916e8ced5d45a7a.lock


Using cache found in /root/.cache/torch/hub/s3prl_s3prl_master
/root/anaconda3/lib/python3.7/site-packages/torchaudio/functional.py:318: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  "At least one mel filterbank has all zero values. "


[UpstreamExpert] - Using the default upstream expert config


In [32]:
loaders["new_test"] = DataLoader(
        newtest_ds,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        drop_last=True,
    )
DEVICE = 'cuda'
model = model.to(DEVICE)

In [33]:
true_labels = []
predicted_labels = []
for batch in tqdm(loaders["new_test"]):
    model.eval()
    prediction = model(batch["representation"].to(DEVICE)).detach().cpu()
    predicted_labels.extend(torch.argmax(prediction, dim=1).tolist())
    true_labels.extend(batch["target"].tolist())


100%|██████████| 4506/4506 [42:57<00:00,  1.75it/s]  


In [34]:
from sklearn.metrics import classification_report

report = classification_report(true_labels, predicted_labels, target_names=list(targets.keys()), labels=range(4))
print(report)



              precision    recall  f1-score   support

          kz       0.39      1.00      0.56     13941
          ru       0.00      0.00      0.00     12107
          en       0.04      0.00      0.00     10000
       other       0.00      0.00      0.00         0

    accuracy                           0.39     36048
   macro avg       0.11      0.25      0.14     36048
weighted avg       0.16      0.39      0.22     36048



/root/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
